In [60]:
import feedparser
from afinn import Afinn
import pandas as pd

In [68]:
feed = feedparser.parse("http://politiken.dk/rss/senestenyt.rss")
afinn = Afinn (language = "da", emoticons = True)
data_titles = []
data_scores = []
for i in range(len(feed["entries"])):
    data_titles.append(feed["entries"][i]["title"])
    data_scores.append(afinn.score(feed["entries"][i]["title"]))

pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame({"Title": data_titles, "Score": data_scores})
df

,Title,Score
0,Forskning i laserfysik giver nobelpris,0.0
1,Frankrig knytter Iran til mislykket angreb i Paris,-1.0
2,Regeringen vil stoppe for salg af nye diesel- og benzinbiler i 2030,-1.0
3,Tyrkiet anholder flere hundrede for at sende valuta ud af landet,0.0
4,Peter Møller ansat som fodbolddirektør i DBU,0.0
5,"Hyldest fra de nærmeste: »Kære Kim, jeg er så ked af, at du ikke ringer til mig mere«",3.0
6,Støjberg er klar til at præsentere dansk-østrigsk asylplan,1.0
7,»Det næste niveau«: Benal tager på miniturné til danske stadioner,0.0
8,"Christian Graugaard: Da Kim Larsen sagde farvel, sluttede min barndom med et brag",0.0
9,Ellen Trane: Butikker kan miste retten til tobakssalg,-2.0
